# RuleFit Analysis

The goal of the notebook is to use machine learning to automatically create a set of rules to predict a target column.  The rules provide an explainable solution.

## Steps

1. Import and clean data
2. Train rule-fit model
3. Examine rules

## Step 1. Import and Clean Data

In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 min 51 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,"14 days, 20 hours and 3 minutes"
H2O cluster name:,H2O_from_python_megankurka_wcjgm7
H2O cluster total nodes:,1
H2O cluster free memory:,4.000 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"locked, healthy"


In [2]:
df = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv",
                    col_types = {'pclass': "enum", 'survived': "enum"})
df.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


## Step 2: Train Rule-Fit Model

We will train a rule-fit model to predict the survival.  The outcome of the rulefit model is rules defining whether or not someone will survive.  The rulefit model is done with the following steps:

1. Train a series of random forest models with different depths
2. Extract rules from the random forest models
3. Train a GLM model with Lasso regularization using the rules to predict the target. 
4. Extract the most important rules.

In [3]:
from rulefit import H2ORuleFit

x =  ["age", "sibsp", "parch", "fare", "sex", "pclass"]
rulefit_model = H2ORuleFit(seed = 1234)
rulefit_model.train(training_frame = df, x = x, y = "survived")

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%


In [4]:
print("Intercept: " + str(round(rulefit_model.intercept.get("Intercept"), 10)))
print("\n\n")

for i in range(len(rulefit_model.rule_importance)):
    print("Coefficient:" + str(round(rulefit_model.rule_importance.iloc[i]["coefficient"], 15)) 
          + "\nRule: " + rulefit_model.rule_importance.iloc[i]["rule"] + "\n\n")

Intercept: -0.1240236316



Coefficient:-0.561494513643133
Rule: age >= 9.522 AND sex is in male


Coefficient:-0.23169993531624
Rule: pclass is in 2, 3 AND age >= 9.522 AND sex is in male


Coefficient:0.195494369282172
Rule: sibsp < 2.5 AND sex is in female


Coefficient:0.102910087408865
Rule: pclass is in 1, 2 AND sibsp < 3.5 AND sex is in female


Coefficient:-0.025038210943377
Rule: age >= 13.498 AND sex is in male


Coefficient:0.004608610632822
Rule: pclass is in 1, 2 AND sex is in female




There are only 5 rules that are created which are recapped below: 

**Highest Likelihood of Survival**

1. Women in class 1 or 2
2. Women with 2 siblings + spouses or less

**Lowest Likelihood of Survival**
1. Men age 9+
2. Men age 10+ in class 2 or 3
3. Men 6+ in class 2 or 3 with fare < $52

Note: The rules are additive.  That means that if a passenger is described by multiple rules, their probability is added together from those rules.

In [60]:
h2o.cluster().shutdown()

H2O session _sid_abec closed.
